<a href="https://colab.research.google.com/github/NID123-CH/NLP--Hugging-Face-Project/blob/main/DSR_40_Language_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import random
import shutil
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras import models, layers
from tqdm import tqdm

In [ ]:
# Where the text files are going to live.
dataset_path = "dataset"
dataset_path_all = os.path.join(dataset_path, "all")
dataset_path_train = os.path.join(dataset_path, "train")
dataset_path_valid = os.path.join(dataset_path, "valid")

# Just use 20 files.
file_number = 100

# Gather the corpus if it has not been gathered yet.
if not os.path.exists(dataset_path):

    # Create all the folders.
    for path in [dataset_path, dataset_path_all, dataset_path_train, dataset_path_valid]:
        if not os.path.exists(path):
            os.mkdir(path)

    # Clone the repo.
    !git clone https://github.com/vilmibm/lovecraftcorpus

    # Find all the files.
    paths_all = glob.glob("lovecraftcorpus/*.txt")
    print(sorted(paths_all))

    # Standardize.
    for path in paths_all:
        content = open(path).read()
        content = content.lower()
        for punctuation in ".,:;?!":
            content = content.replace(punctuation, " " + punctuation)
        open(path, "w").write(content)

    # Do not use all.
    paths_all = paths_all[:file_number]

    # Split 80/20.
    split_index = int(len(paths_all) * 0.8)
    paths_train = paths_all[:split_index]
    paths_valid = paths_all[split_index:]

    # Copy files.
    def copy(paths, destination):
        for path in paths:
            shutil.copy2(path, destination)
    copy(paths_all, dataset_path_all)
    copy(paths_train, dataset_path_train)
    copy(paths_valid, dataset_path_valid)

    # Delete repo.
    !rm -rf lovecraftcorpus

    # Done.
    print("Corpus downloaded.")

Cloning into 'lovecraftcorpus'...
remote: Enumerating objects: 74, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 74 (delta 0), reused 3 (delta 0), pack-reused 70 (from 1)
Receiving objects: 100% (74/74), 1.12 MiB | 3.54 MiB/s, done.
['lovecraftcorpus/alchemist.txt', 'lovecraftcorpus/arthur_jermyn.txt', 'lovecraftcorpus/azathoth.txt', 'lovecraftcorpus/beast.txt', 'lovecraftcorpus/beyond_wall_of_sleep.txt', 'lovecraftcorpus/book.txt', 'lovecraftcorpus/celephais.txt', 'lovecraftcorpus/charles_dexter_ward.txt', 'lovecraftcorpus/clergyman.txt', 'lovecraftcorpus/colour_out_of_space.txt', 'lovecraftcorpus/cool_air.txt', 'lovecraftcorpus/crawling_chaos.txt', 'lovecraftcorpus/cthulhu.txt', 'lovecraftcorpus/dagon.txt', 'lovecraftcorpus/descendent.txt', 'lovecraftcorpus/doorstep.txt', 'lovecraftcorpus/dreams_in_the_witch.txt', 'lovecraftcorpus/dunwich.txt', 'lovecraftcorpus/erich_zann.txt', 'lovecraftcorpus/ex_oblivione.txt', 'lovec

# Data

In [ ]:
batch_size = 32 # Not for SGD!
seed = 42

def create_dataset(dataset_path):
    dataset = preprocessing.text_dataset_from_directory(
        dataset_path,
        labels=None,
        batch_size=batch_size,
        seed=seed
    )
    return dataset

dataset_original_all   = create_dataset(dataset_path_all)
dataset_original_train = create_dataset(dataset_path_train)
dataset_original_valid = create_dataset(dataset_path_valid)

Found 67 files.
Found 53 files.
Found 14 files.


In [ ]:
for x in dataset_original_all.take(1):
    print(x.shape)
    print(x[0])

(32,)
tf.Tensor(b'the call of cthulhu\n\nof such great powers or beings there may be conceivably a survival . . .a survival of a hugely remote period when . . .consciousness was manifested , perhaps , in shapes and forms long since withdrawn before the tide of advancing humanity . . .forms of which poetry and legend alone have caught a flying memory and called them gods , monsters , mythical beings of all sorts and kinds . . .\n\n--algernon blackwood\n\ni . the horror in clay\n\nthe most merciful thing in the world , i think , is the inability of the human mind to correlate all its contents . we live on a placid island of ignorance in the midst of black seas of infinity , and it was not meant that we should voyage far . the sciences , each straining in its own direction , have hitherto harmed us little ; but some day the piecing together of dissociated knowledge will open up such terrifying vistas of reality , and of our frightful position therein , that we shall either go mad from the

# Create the text vectorizer.

In [ ]:
vocabulary_size = 10_000

encoder = layers.TextVectorization(
    max_tokens=vocabulary_size,
    standardize=None,
    split="whitespace",
    output_mode="int"
)
encoder.adapt(dataset_original_all)

vocabulary = encoder.get_vocabulary()
print(vocabulary)

['', '[UNK]', 'the', ',', 'and', 'of', '.', 'to', 'a', 'in', 'i', 'was', 'that', 'had', 'he', 'it', 'as', 'with', 'his', ';', 'for', 'at', 'which', 'from', 'on', 'not', 'but', 'my', 'were', 'by', 'they', 'all', 'be', 'or', 'this', 'there', 'me', 'have', 'could', 'one', 'when', 'him', 'no', 'an', 'we', 'been', 'some', 'what', 'their', 'so', 'would', 'is', 'its', 'more', 'out', 'old', 'them', 'only', 'now', 'about', 'up', 'did', 'into', 'before', 'seemed', 'very', 'then', 'than', 'great', 'those', 'through', 'time', 'who', 'after', 'any', 'saw', 'where', 'even', 'you', 'things', 'though', 'must', 'our', 'over', 'if', 'down', 'other', 'man', 'like', 'these', 'came', 'found', 'might', 'such', 'whose', 'are', 'night', 'upon', 'strange', 'never', 'thing', '?', 'once', 'made', 'much', 'place', 'still', 'long', 'men', 'certain', 'most', 'knew', 'beyond', 'first', 'many', 'carter', 'see', 'come', 'thought', 'house', 'heard', 'black', 'two', 'last', 'too', 'yet', 'seen', 'back', 'again', 'told',

# Create dataset for autoregression.

In [ ]:
sequence_length = 512
padding_token_id = 0

def create_dataset_for_autoregression(dataset, hop_length=1):
    x_inputs = []
    y_outputs = []

    for books in dataset:

        books = encoder(books).numpy() # Encodes all books into word indices.

        for book in books:

            # Remove the padding tokens.
            book = [index for index in list(book) if index != padding_token_id]

            for start_index in range(0, len(book) - sequence_length, hop_length):
                x = book[start_index:start_index + sequence_length]
                y = book[start_index + 1: start_index + sequence_length + 1]
                assert len(x) == sequence_length
                assert len(y) == sequence_length

                x_inputs += [x]
                y_outputs += [y]

    return tf.data.Dataset.from_tensor_slices((x_inputs, y_outputs))


dataset_train = create_dataset_for_autoregression(dataset_original_train)
dataset_valid = create_dataset_for_autoregression(dataset_original_valid)

In [ ]:
def decode(indices):
    return " ".join([vocabulary[index] for index in indices])

for input, output in dataset_train.take(8):
    print("in: ", decode(input))
    print("out:", decode(output))
    print("")

in:  the case of charles dexter ward the essential saltes of animals may be so prepared and preserved , that an ingenious man may have the whole [UNK] of [UNK] in his own [UNK] , and raise the fine shape of an animal out of its ashes at his pleasure ; and by the [UNK] method from the essential saltes of [UNK] dust , a philosopher may , without any [UNK] necromancy , call up the shape of any dead [UNK] from the dust [UNK] his [UNK] has been incinerated . [UNK] contents i . a result and a prologue ii . an [UNK] and a horror iii . a search and an [UNK] iv . a mutation and a madness v . a nightmare and a cataclysm i . a result and a prologue 1 from a private hospital for the insane near providence , rhode island , there recently disappeared an exceedingly singular person . he bore the name of charles dexter ward , and was placed under restraint most reluctantly by the [UNK] father who had watched his [UNK] grow from a mere eccentricity to a dark mania involving both a possibility of murder

In [ ]:
import matplotlib.pyplot as plt

def render_history(history):
    plt.title("Training loss vs. validation loss")
    plt.plot(history.history["loss"], label="loss")
    plt.plot(history.history["val_loss"], label="val_loss")
    plt.legend()
    plt.show()
    plt.close()

    plt.title("Training accuracy vs. validation accuracy")
    plt.plot(history.history["accuracy"], label="accuracy")
    plt.plot(history.history["val_accuracy"], label="val_accuracy")
    plt.legend()
    plt.show()
    plt.close()

## Train the model

In [ ]:
embedding_size = 128

model = models.Sequential()
model.add(layers.Embedding(vocabulary_size, embedding_size)) # Like is!
#model.add(layers.Embedding(vocabulary_size, embedding_size, input_length=sequence_length))

model.add(layers.Dropout(0.1)) # Plot twist!
model.add(layers.LSTM(embedding_size, return_sequences=True)) # Plot twist!
model.add(layers.Dropout(0.1)) # Plot twist!
model.add(layers.Dense(vocabulary_size, activation="softmax"))
model.summary()

model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy", # Important! Computing the error between a predicted prob-dist and the label.
    metrics=["accuracy"]
)

model.fit(
    dataset_train.shuffle(10_000).batch(128),
    epochs=10, # TODO go up to 10
    validation_data=dataset_valid.batch(128)
)

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Epoch 1/10
3285/3285 ━━━━━━━━━━━━━━━━━━━━ 1118s 340ms/step - accuracy: 0.1383 - loss: 5.4518 - val_accuracy: 0.1540 - val_loss: 6.1989
Epoch 2/10
 744/3285 ━━━━━━━━━━━━━━━━━━━━ 12:22 292ms/step - accuracy: 0.2452 - loss: 4.3628

In [ ]:
model.summary()

render_history(model.history)

In [ ]:
import numpy as np

def generate(model, seed_text, generated_sequence_length, temperature):

    seed_text = seed_text.lower()
    for punctuation in ".,:;?!":
        seed_text = seed_text.replace(punctuation, " " + punctuation)
    print(seed_text)

    input_sequence = encoder(seed_text).numpy().tolist()
    print(input_sequence)

    # Generate the sequence by repeatedly predicting.
    while len(input_sequence) < generated_sequence_length:
        prediction = model.predict(np.expand_dims(input_sequence, axis=0), verbose=False)
        #print(prediction.shape)
        #plt.plot(prediction[0][-1])
        #assert False
        predicted_index = get_index_from_prediction(prediction[0][-1], temperature)
        input_sequence.append(predicted_index)

    # Convert the generated sequence to a string.
    text = decode(input_sequence)
    for punctuation in ".,:;?!":
        text = text.replace(" " + punctuation, punctuation)
    print(text)
    print("")


def get_index_from_prediction(prediction, temperature=0.0):
    """ Gets an index from a prediction. """

    # Zero temperature - use the argmax.
    if temperature == 0.0:
        return np.argmax(prediction)

    # Non-zero temperature - do some random magic.
    else:
        prediction = np.asarray(prediction).astype('float64')
        prediction = np.log(prediction) / temperature
        exp_prediction= np.exp(prediction)
        prediction = exp_prediction / np.sum(exp_prediction)
        probabilities = np.random.multinomial(1, prediction, 1)
        return np.argmax(probabilities)


generate(model, "The Shadow out of time", 100, temperature=1.0)